In [1]:
from chess_env import ChessEnv
from network import Agent

env = ChessEnv()
agent = Agent(
    gamma=0.99,
    epsilon=1.0,
    batch_size=128,
    max_me_size=1_000,
    eps_end=0.01,
    input_dims=[390, 390, 3],
    lr=0.003
)
scores = []
games = {}
number_of_games = 20

agent.Q_eval.device

device(type='cpu')

In [2]:
for game in range(number_of_games):
    score = 0
    observation, info = env.reset()
    while not env.game.is_game_over():
        current_action_space = env.action_space
        action = agent.choose_action(observation, current_action_space)
        new_observation, reward, ended, _, info = env.step(action)
        score += reward
        agent.store_transition(
            observation,
            current_action_space.index(action),
            reward,
            new_observation,
            ended
        )
        agent.learn(current_action_space, env.action_space)
        observation = new_observation

    games[env.pgn()] = {
        "scores" : scores,
        "eps_hist" : agent.epsilon
    }

    print(f"episode: {game}, score: {score:.2f}, epsilon: {agent.epsilon:.2f}")

episode: 0, score: -374.00, epsilon: 1.00
torch.Size([128, 9, 390, 36]) (128,) (128, 390, 390, 3)


IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [128], [128, 390, 390, 3]